In [1]:
import numpy as np
from tensorflow import keras

import utils

In [2]:
(train_images, train_labels), (test_images, test_labels) = keras.datasets.fashion_mnist.load_data()

train_images  = train_images / 255.0
train_images = np.expand_dims(train_images, axis=-1)
print(f"train_images.shape = {train_images.shape}, train_labels.shape = {train_labels.shape}")

test_images = test_images / 255.0
test_images = np.expand_dims(test_images, axis=-1)
print(f"test_images.shape = {test_images.shape}, test_labels.shape = {test_labels.shape}")

train_images.shape = (60000, 28, 28, 1), train_labels.shape = (60000,)
test_images.shape = (10000, 28, 28, 1), test_labels.shape = (10000,)


In [3]:
def split(images, labels):
    split_mask = np.isin(labels, [6, 7])
    images_a = images[~split_mask]
    labels_a = labels[~split_mask]
    labels_a[labels_a > 7] -= 2

    images_b = images[split_mask]
    labels_b = labels[split_mask]
    labels_b = labels_b - 6

    return (images_a, labels_a), (images_b, labels_b)

In [4]:
(train_images_a, train_labels_a), (train_images_b, train_labels_b) = split(train_images, train_labels)
train_images_b = train_images_b[:8]
train_labels_b = train_labels_b[:8]
print(f"train_images_a.shape = {train_images_a.shape}, train_images_b.shape = {train_images_b.shape}")

(test_images_a, test_labels_a), (test_images_b, test_labels_b) = split(test_images, test_labels)
print(f"test_images_a.shape = {test_images_a.shape}, test_images_b.shape = {test_images_b.shape}")

train_images_a.shape = (48000, 28, 28, 1), train_images_b.shape = (8, 28, 28, 1)
test_images_a.shape = (8000, 28, 28, 1), test_images_b.shape = (2000, 28, 28, 1)


## Model A

In [5]:
utils.reset_session()

model_a = keras.models.Sequential([
    keras.layers.Conv2D(64, (3, 3), activation='relu', input_shape=(28, 28, 1)),
    keras.layers.MaxPooling2D(2, 2),
    keras.layers.Conv2D(64, (3, 3), activation='relu'),
    keras.layers.MaxPooling2D(2, 2),
    keras.layers.Flatten(),
    keras.layers.Dense(128, activation='relu'), 
    keras.layers.Dense(8, activation='softmax')
])


optimizer = keras.optimizers.Adam()
model_a.compile(
    optimizer=optimizer,
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

history = model_a.fit(
    train_images_a, train_labels_a, 
    epochs=10, 
    batch_size=64,
    validation_data=(test_images_a, test_labels_a)
)

model_a.save('model_a.h5')

Epoch 1/10
750/750 [==============================] - 3s 3ms/step - loss: 0.3047 - accuracy: 0.8975 - val_loss: 0.2220 - val_accuracy: 0.9261
Epoch 2/10
750/750 [==============================] - 2s 3ms/step - loss: 0.1825 - accuracy: 0.9375 - val_loss: 0.1685 - val_accuracy: 0.9434
Epoch 3/10
750/750 [==============================] - 2s 3ms/step - loss: 0.1508 - accuracy: 0.9479 - val_loss: 0.1519 - val_accuracy: 0.9515
Epoch 4/10
750/750 [==============================] - 2s 3ms/step - loss: 0.1310 - accuracy: 0.9541 - val_loss: 0.1515 - val_accuracy: 0.9496
Epoch 5/10
750/750 [==============================] - 2s 3ms/step - loss: 0.1134 - accuracy: 0.9585 - val_loss: 0.1413 - val_accuracy: 0.9526
Epoch 6/10
750/750 [==============================] - 2s 3ms/step - loss: 0.0988 - accuracy: 0.9647 - val_loss: 0.1520 - val_accuracy: 0.9456
Epoch 7/10
750/750 [==============================] - 2s 3ms/step - loss: 0.0868 - accuracy: 0.9685 - val_loss: 0.1367 - val_accuracy: 0.9553
Epoch 

## Model B

In [6]:
utils.reset_session()

model_b = keras.models.Sequential([
    keras.layers.Conv2D(64, (3, 3), activation='relu', input_shape=(28, 28, 1)),
    keras.layers.MaxPooling2D(2, 2),
    keras.layers.Conv2D(64, (3, 3), activation='relu'),
    keras.layers.MaxPooling2D(2, 2),
    keras.layers.Flatten(),
    keras.layers.Dense(128, activation='relu'), 
    keras.layers.Dense(1, activation='sigmoid')
])


optimizer = keras.optimizers.Adam()
model_b.compile(
    optimizer=optimizer,
    loss='binary_crossentropy',
    metrics=['accuracy']
)

history = model_b.fit(
    train_images_b, train_labels_b, 
    epochs=5, 
    batch_size=8,
    validation_data=(test_images_b, test_labels_b)
)

model_b.save('model_b.h5')

Epoch 1/5
1/1 [==============================] - 1s 506ms/step - loss: 0.6742 - accuracy: 0.6250 - val_loss: 0.6292 - val_accuracy: 0.5000
Epoch 2/5
1/1 [==============================] - 0s 417ms/step - loss: 0.5771 - accuracy: 0.6250 - val_loss: 0.5815 - val_accuracy: 0.5000
Epoch 3/5
1/1 [==============================] - 0s 411ms/step - loss: 0.4957 - accuracy: 0.7500 - val_loss: 0.5254 - val_accuracy: 0.5970
Epoch 4/5
1/1 [==============================] - 0s 481ms/step - loss: 0.4153 - accuracy: 0.8750 - val_loss: 0.4446 - val_accuracy: 0.8315
Epoch 5/5
1/1 [==============================] - 0s 444ms/step - loss: 0.3283 - accuracy: 1.0000 - val_loss: 0.3446 - val_accuracy: 0.9465


## Model B on A

In [7]:
model_a = keras.models.load_model('model_a.h5')

model_b_on_a = keras.models.Sequential(model_a.layers[:-1])
model_b_on_a.add(keras.layers.Dense(1, activation='sigmoid'))

# for layer in model_b_on_a.layers[:-1]:
#     layer.trainable = False

optimizer = keras.optimizers.Adam()
model_b_on_a.compile(
    optimizer=optimizer,
    loss='binary_crossentropy',
    metrics=['accuracy']
)

history = model_b_on_a.fit(
    train_images_b, train_labels_b, 
    epochs=5, 
    batch_size=8,
    validation_data=(test_images_b, test_labels_b)
)

Epoch 1/5
1/1 [==============================] - 1s 546ms/step - loss: 1.4139 - accuracy: 0.3750 - val_loss: 0.5966 - val_accuracy: 0.6735
Epoch 2/5
1/1 [==============================] - 0s 481ms/step - loss: 0.7846 - accuracy: 0.3750 - val_loss: 0.3431 - val_accuracy: 0.8320
Epoch 3/5
1/1 [==============================] - 0s 400ms/step - loss: 0.3817 - accuracy: 0.7500 - val_loss: 0.1879 - val_accuracy: 0.9660
Epoch 4/5
1/1 [==============================] - 0s 357ms/step - loss: 0.1665 - accuracy: 1.0000 - val_loss: 0.1030 - val_accuracy: 0.9935
Epoch 5/5
1/1 [==============================] - 0s 372ms/step - loss: 0.0694 - accuracy: 1.0000 - val_loss: 0.0603 - val_accuracy: 0.9975
